# From QuPath to Visualisation and Quantification

NOTE: For the sake of speed, we are using the 25 micron atlas. In a real case scenario, it's much better to use the 10 micron atlas. Simply replace `allen_mouse_25um` by `allen_mouse_10um`.

## Collecting and filtering results 

After detecting the cells in QuPath, you have exported the results as tables, saved as csv files. In the QuPath project folder you'll find a `results` folder that contains `Image1Name_detections.csv` , `Image1Name_annotations.csv` ... 

Let's see a way to visualise and process these data.



### Merging all data into a single dataframe

Let's merge all csv of cell detections.

In [1]:
# Import dependencies
import os
import glob
import pandas as pd
import numpy as np

#from bg_atlasapi.bg_atlas import BrainGlobeAtlas
verbose = True

Specify the folder where csv files can be found

In [10]:
dir = "/home/gabri/Desktop/test_abba/587/results"

detections_csv_files = glob.glob(dir+'/*measurements.tsv')
if (verbose) : print('\n'.join(detections_csv_files[:5]))

df = []

# here you can use detections_csv_files[:10] to process only few files and go faster
for file in detections_csv_files : 
    df.append( pd.read_csv(file , sep='\t', header=(0) ) )
    
big_frame = pd.concat(df, ignore_index=True)

# Here we'll keep only the image, the parent region, coordinates and Mean Fluorescent value in Cy3 channel
filtered_df = big_frame[ ['Image', 'Parent', 'Atlas_X', 'Atlas_Y', 'Atlas_Z', 'Cell: CY3 mean']]
filtered_df[:] # Display a subset of cells to check that we collected the right columns

cy3_filtered_df = filtered_df[filtered_df[ 'Cell: CY3 mean'] > 100 ]

cy3_pos_cells = np.array( cy3_filtered_df[['Atlas_X', 'Atlas_Y','Atlas_Z']] , float )#* 1000.0

/home/gabri/Desktop/test_abba/587/results/measurements.tsv


Make a list of all csv files containing detections

In [3]:
detections_csv_files = glob.glob(dir+'/*measurements.tsv')
if (verbose) : print('\n'.join(detections_csv_files[:5]))

detections_csv_files

/home/gabri/Desktop/test_abba/587/results/measurements.tsv


['/home/gabri/Desktop/test_abba/587/results/measurements.tsv']

Append the content of the files in a big frame and select the columns we want to keep

In [4]:
df = []

# here you can use detections_csv_files[:10] to process only few files and go faster
for file in detections_csv_files : 
    df.append( pd.read_csv(file , sep='\t', header=(0) ) )
    
big_frame = pd.concat(df, ignore_index=True)

# Here we'll keep only the image, the parent region, coordinates and Mean Fluorescent value in Cy3 channel
filtered_df = big_frame[ ['Image', 'Parent', 'Atlas_X', 'Atlas_Y', 'Atlas_Z', 'Cell: CY3 mean']]
filtered_df[:] # Display a subset of cells to check that we collected the right columns

,Image,Parent,Atlas_X,Atlas_Y,Atlas_Z,Cell: CY3 mean
0,587_FB_4B_01.vsi - 10x_02,Annotation,0.2156,-60.9867,103.2067,427.1538
1,587_FB_4B_01.vsi - 10x_02,Annotation,0.2157,-60.9903,103.2061,421.6250
2,587_FB_4B_01.vsi - 10x_02,Annotation,0.2173,-61.0623,103.2662,668.3333
3,587_FB_4B_01.vsi - 10x_02,Annotation,0.2208,-61.1988,103.2806,552.9048
4,587_FB_4B_01.vsi - 10x_02,Annotation,0.2174,-61.0813,103.3337,517.4091
...,...,...,...,...,...,...
6338,587_FB_4B_01.vsi - 10x_02,Annotation,0.0901,-56.8852,106.7260,539.0556
6339,587_FB_4B_01.vsi - 10x_02,Annotation,0.0911,-56.9250,106.7408,490.0635
6340,587_FB_4B_01.vsi - 10x_02,Annotation,0.0901,-56.8885,106.7408,403.5217
6341,587_FB_4B_01.vsi - 10x_02,Annotation,0.0961,-57.1305,106.8005,583.9474


### Basic data filtering according to a property

Filter data and keep cells for which the mean value of the Cy3 channel is above 100 values for cell above 100

In [5]:
cy3_filtered_df = filtered_df[filtered_df[ 'Cell: CY3 mean'] > 100 ]
cy3_filtered_df[:10]

,Image,Parent,Atlas_X,Atlas_Y,Atlas_Z,Cell: CY3 mean
0,587_FB_4B_01.vsi - 10x_02,Annotation,0.2156,-60.9867,103.2067,427.1538
1,587_FB_4B_01.vsi - 10x_02,Annotation,0.2157,-60.9903,103.2061,421.6250
2,587_FB_4B_01.vsi - 10x_02,Annotation,0.2173,-61.0623,103.2662,668.3333
3,587_FB_4B_01.vsi - 10x_02,Annotation,0.2208,-61.1988,103.2806,552.9048
4,587_FB_4B_01.vsi - 10x_02,Annotation,0.2174,-61.0813,103.3337,517.4091
5,587_FB_4B_01.vsi - 10x_02,Annotation,0.2151,-61.0023,103.3827,380.9688
6,587_FB_4B_01.vsi - 10x_02,Annotation,0.2262,-61.4245,103.3837,691.8800
7,587_FB_4B_01.vsi - 10x_02,Annotation,0.2274,-61.4765,103.4148,671.2000
8,587_FB_4B_01.vsi - 10x_02,Annotation,0.2280,-61.5033,103.4257,623.2500
9,587_FB_4B_01.vsi - 10x_02,Annotation,0.2281,-61.5069,103.4272,704.5946


### Converting Allen CCFv3 coordinates to BrainGlobe compatible coordinates

In [6]:
# Converts the coordinates to python and brainglobe expectations: swap X and Z, converts from millimeters to micrometers
cy3_pos_cells = np.array( cy3_filtered_df[['Atlas_X', 'Atlas_Y','Atlas_Z']] , float ) * 1000.0

cy3_pos_cells[cy3_pos_cells[:,2]>90000,2] -= 20000

cy3_pos_cells /1000


viewer.add_points( cy3_pos_cells, 
                      size = 50, 
                      name = "Cy3+ Cells",
                      blending = 'additive' # the default blending is not ideal when so many points are displayed in 3D
                 )


NameError: name 'viewer' is not defined

In [2]:
dir = "/home/gabri/Desktop/test_abba/587/results"

detections_csv_files = glob.glob(dir+'/*measurements.tsv')
if (verbose) : print('\n'.join(detections_csv_files[:5]))

df = []

# here you can use detections_csv_files[:10] to process only few files and go faster
for file in detections_csv_files : 
    df.append( pd.read_csv(file , sep='\t', header=(0) ) )
    
big_frame = pd.concat(df, ignore_index=True)

# Here we'll keep only the image, the parent region, coordinates and Mean Fluorescent value in Cy3 channel
filtered_df = big_frame[ ['Image', 'Parent', 'Atlas_X', 'Atlas_Y', 'Atlas_Z', 'Cell: CY3 mean']]
filtered_df[:] # Display a subset of cells to check that we collected the right columns

cy3_filtered_df = filtered_df[filtered_df[ 'Cell: CY3 mean'] > 100 ]

cy3_pos_cells = np.array( cy3_filtered_df[['Atlas_X', 'Atlas_Y','Atlas_Z']] , float ) * 1000

print(cy3_pos_cells)

#create lists from cells table - note swapping X and Y axes to match brainrender orientation
Y = cy3_filtered_df['Atlas_X'].tolist()#200
X = cy3_filtered_df['Atlas_Y'].tolist()#-6000
Z = cy3_filtered_df['Atlas_Z'].tolist()#10000

AtlasRes = 1000
X = [element * AtlasRes for element in X]
Y = [element * AtlasRes for element in Y]
Z = [element * AtlasRes for element in Z]
pts = [[x, y, z] for x, y, z in zip(X, Y, Z)]
coordinates = np.vstack(pts)

print("\n\n")
print(coordinates)


R_x = np.array([
    [1, 0, 0],
    [0, 0, -1],
    [0, 1, 0]
])

R_y = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [-1, 0, 0]
])

R_z = np.array([
    [0, -1, 0],
    [1, 0, 0],
    [0, 0, 1]
])

#coordinates = coordinates @ R_z.T



# Specify the file name
#import csv
#filename = "/home/gabri/Desktop/test_abba/587/results/numbers_output.csv"
#with open(filename, mode='w', newline='') as file:
#    writer = csv.writer(file)
#    writer.writerows(cy3_pos_cells)


import numpy as np
# Necessary for running in jupyter
import vedo
vedo.settings.default_backend= 'vtk'

from brainrender import Scene
from brainrender.actors import Points

# Display the Allen Brain mouse atlas.
scene = Scene(atlas_name="allen_mouse_50um", title="Cells in primary visual cortex")

# Display a brain region
#primary_visual = scene.add_brain_region("VISp", alpha=0.2)

# Create a Points actor
cells = Points(coordinates)

# Add to scene
scene.add(cells)


# Add to scene
scene.add(Points(np.array([[0,0,0]]), radius=1000))


# Add label to the brain region
#scene.add_label(primary_visual, "Primary visual cortex")

# Display the figure.
scene.render()

/home/gabri/Desktop/test_abba/587/results/measurements.tsv
[[ 2.156000e+02 -6.098670e+04  1.032067e+05]
 [ 2.157000e+02 -6.099030e+04  1.032061e+05]
 [ 2.173000e+02 -6.106230e+04  1.032662e+05]
 ...
 [ 9.010000e+01 -5.688850e+04  1.067408e+05]
 [ 9.610000e+01 -5.713050e+04  1.068005e+05]
 [ 9.250000e+01 -5.699680e+04  1.068184e+05]]



[[-6.098670e+04  2.156000e+02  1.032067e+05]
 [-6.099030e+04  2.157000e+02  1.032061e+05]
 [-6.106230e+04  2.173000e+02  1.032662e+05]
 ...
 [-5.688850e+04  9.010000e+01  1.067408e+05]
 [-5.713050e+04  9.610000e+01  1.068005e+05]
 [-5.699680e+04  9.250000e+01  1.068184e+05]]


In [104]:
import numpy as np
# Necessary for running in jupyter
import vedo
vedo.settings.default_backend= 'vtk'

from brainrender import Scene
from brainrender.actors import Points

# Display the Allen Brain mouse atlas.
scene = Scene(atlas_name="allen_mouse_25um", title="Cells in primary visual cortex")

# Display a brain region
primary_visual = scene.add_brain_region("VISp", alpha=0.2)

# Create a Points actor
cells = Points(coordinates)

# Add to scene
scene.add(cells)

# Add label to the brain region
scene.add_label(primary_visual, "Primary visual cortex")

# Display the figure.
scene.render()

In [117]:
import napari

viewer = napari.Viewer()


viewer.add_points( coordinates , 
                      size = 10 , 
                       name = "ZXY-Mean" ,
                 )
                     

<Points layer 'ZXY-Mean' at 0x7821882587c0>

/home/gabri/mambaforge/envs/brainrender-env/lib/python3.9/site-packages/brainrender_napari/napari_atlas_representation.py:24: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.6.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self._tooltip = QLabel(self.viewer.window.qt_viewer.parent())


### Using Brainglobe to find cells located within an atlas region

We can also filter per regions, that requires a little bit of work: we need to gather all the descendants regions and check whether the detected cells belong to any of the descendant region.

See https://brainglobe.info/index.html for information about BrainGlobe. BrainGlobe is a set of software that eases the manipulation of several atlases with common code. It notably wraps the Allen brain atlas that we are using here.

In [14]:
from bg_atlasapi.bg_atlas import BrainGlobeAtlas

atlas = BrainGlobeAtlas("allen_mouse_25um") # Loads the atlas label image at 25 microns resolution
# see https://github.com/brainglobe/brainglobe-atlasapi/blob/eee08c6d9aa9b69251f5adab3efc2ed85fc91cd8/tutorials/Atlas%20API%20usage.ipynb#L1227


In [27]:
# Define the region we are interested in 
region_of_interest = "AId5"

# get the region ID
region_id = atlas.structures.acronym_to_id_map[ region_of_interest ]
if verbose : print("region_id : " , str(region_id))

# get descendants Names 
descendants_names = atlas.get_structure_descendants(region_id)

# and look for IDs of each one
descendants_ids = []
for name in descendants_names:
    descendants_ids.append( atlas.structures.acronym_to_id_map[name] ) 
    
if verbose : print("descendants_ids : " , descendants_ids)

# Combine IDs of region
region_and_descendants_ids = descendants_ids + [ region_id ]
if verbose : print("region_descendants_ids : ", region_and_descendants_ids )   


region_id :  1101
descendants_ids :  []
region_descendants_ids :  [1101]


In [29]:
# We have all the information to filter the number of detections per region

cells_in_region = []

for cell in cy3_pos_cells:
    structure_id_of_cell = atlas.structure_from_coords(cell, microns=True) # We read the label image at the position of the cell
    if structure_id_of_cell in region_and_descendants_ids:
        cells_in_region.append(cell)

# Number of points in the Region
print("There are "+str(len(cells_in_region))+" cells in the "+region_of_interest+" region.")

IndexError: index -2049 is out of bounds for axis 1 with size 320

## Visualisation of cells in napari

### Create an instance of the napari viewer

In [2]:
%gui qt5
import napari

In [3]:
# Create a Napari viewer in 3D mode
viewer = napari.Viewer(ndisplay=3) # Sets the viewer to 3D mode - the 2D slicing mode is not very convenient here

/home/gabri/mambaforge/envs/brainrender-env/lib/python3.9/site-packages/brainrender_napari/napari_atlas_representation.py:24: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.6.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self._tooltip = QLabel(self.viewer.window.qt_viewer.parent())


### Display all cells as a point cloud

In [61]:
# Display all positive detected cells
viewer.add_points( cy3_pos_cells, 
                      size = 50, 
                      name = "Cy3+ Cells",
                      blending = 'additive' # the default blending is not ideal when so many points are displayed in 3D
                 )

# You can lower the opacity of the layer to see a bit better the structure of the points (0.10).

<Points layer 'Cy3+ Cells' at 0x7c8bdae6d130>

### Display all cells colored according to a property

In [27]:
# We can color the points according to a property of the dataframe, for instance according the the mean CY3 intensity

max_cy3 = max(cy3_filtered_df[ 'Cell: CY3 mean'])/3 # the division by 3 is there to increase the level of brightness

color_cells = []

cy3_values = cy3_filtered_df[ 'Cell: CY3 mean'].values # Dataframe to numpy array

# Combine 'cy3' values with constant values for green and blue components
colors_rgb_from_cy3 = [(min(cy3/max_cy3,1), 0, 0,1) for cy3 in cy3_values]  # Assuming cy3 values represent the red component
edge_rgb_from_cy3 = [(min(cy3/max_cy3,1), 0, 0, 0.5) for cy3 in cy3_values]  # Assuming cy3 values represent the red component

# Add colored points layer to Napari viewer
viewer.add_points(cy3_pos_cells, 
                  size=50, 
                  name = "Cy3+ Cells - Cy3 colored",
                  face_color=colors_rgb_from_cy3, 
                  edge_color=edge_rgb_from_cy3, 
                  blending='additive'
                 )


<Points layer 'Cy3+ Cells - Cy3 colored' at 0x77650c7b9cf0>

### Display cells of a subregion

In [30]:
# Display positive cells detected in regions
viewer.add_points( cells_in_region, 
                      size = 25, 
                      symbol = 'diamond',
                      name = "Cy3+ Cells In "+region_of_interest,
                      blending = 'additive',
                      face_color = 'Green',
                      edge_color = 'Green'
                 )

/home/gabri/mambaforge/envs/brainrender-env/lib/python3.9/site-packages/napari/utils/migrations.py:101: FutureWarning: Argument 'edge_color' is deprecated, please use 'border_color' instead. The argument 'edge_color' was deprecated in 0.5.0 and it will be removed in 0.6.0.
  return func(*args, **kwargs)


<Points layer 'Cy3+ Cells In AId5' at 0x7c8c106b6100>

## Visualing atlas regions with brainrender-napari

Brainrender-napari (https://github.com/brainglobe/brainrender-napari) is a napari plugin to render BrainGlobe atlases and associated data as layers. It has a graphical user interface and can also used programmatically.

### With the graphical user interface

Within napari, you can:
- click on `Plugins > Brainrender`. This will take a few minutes. But then a side bar will show up.
- click on `Allen mouse (25 um)`

You can now navigate through the atlas hierarchy. Double click on any of the region (like root), and then it will show up as a mesh in the viewer.

See https://brainglobe.info/tutorials/visualise-atlas-napari.html for other functionalities

### With the application programming interface

In [ ]:
# See documentation at https://github.com/brainglobe/brainrender-napari

# NOT POSSIBLE YET! See https://github.com/brainglobe/brainrender-napari/issues/53

## Counting cells per region

### Dict: id to cell count, acronym to cell count

Here we count the cells and sort them by region. Each region has certain properties, like an id, which is a unique number in the atlas, and an acronym.

In the cell below, we sort all cells by region id (in contrast to the cells above, a cell belongs to one region only - this means that if we sum all cells from all regions, we will find the total number of detected cells). Be aware that if you find cells in a region which is not in a leaf of the atlas, this means that the cell is in the region but NOT in the subregions.

In the cell below, we sort all cells by id, and then convert this data to a more human readable dict where each key is the acronym of the region instead of the int id.

In [4]:
from collections import defaultdict

# Create a defaultdict with int as the default factory
count_per_id = defaultdict(int)

for cell in cy3_pos_cells:
    structure_id_of_cell = atlas.structure_from_coords(cell, microns=True) # We read the label image at the position of the cell
    count_per_id[structure_id_of_cell] += 1

# We can convert this dict to a count per acronym

count_per_acronym = {}

for struct_id, count in count_per_id.items():
    structure_info = atlas.structures.get(struct_id)
    if structure_info:
        acronym = structure_info['acronym']
        count_per_acronym[acronym] = count
    else:
        # Handle the case where structure ID is not found
        # For example, you can skip it or assign a default value
        count_per_acronym['OUTSIDE'] = count_per_acronym.get('OUTSIDE',0) + 1

print(count_per_id)
print(count_per_acronym)


NameError: name 'cy3_pos_cells' is not defined

### Dict: acronym to cell count

In [ ]:
# Or you can directly create a dict with the acronym

count_per_acronym = {}

for cell in cy3_pos_cells:
    try:
        structure_acronym_of_cell = atlas.structure_from_coords(cell, 
                                                           microns=True,
                                                           as_acronym=True) # We read the label image at the position of the cell
        count_per_acronym[structure_acronym_of_cell] = count_per_acronym.get(structure_acronym_of_cell,0) + 1
    except KeyError: # When cells are outside of the Brain, that may happen
        count_per_acronym['OUTSIDE'] = count_per_acronym.get('OUTSIDE',0) + 1


print(count_per_acronym)

### Dict: acronym to cell count, skipping lower levels of the atlas hierarchy

Sometimes you don't want to look at what's happening within the finest subdivisions of the structures. Here's a possible way of cropping data at a certain hierarchy level.

Try running the following cell with different `max_hierarchy_level` (1, then 2, then 10 ...).


In [ ]:
hierarchy_level = len(atlas.structures['GPe']["structure_id_path"]) # Example code: find the path length to get to a certain region of the atlas

max_hierarchy_level = 5 # 1 = root, 2 = VS, fiber tracts, grey, etc.

count_per_acronym_max_level = {}

for cell in cy3_pos_cells:
    struct_id = atlas.structure_from_coords(cell, microns=True) # We read the label image at the position of the cell
    structure_info = atlas.structures.get(struct_id)
    if structure_info:
        struct_id_path = structure_info["structure_id_path"]
        level_hierarchy_chosen = min(max_hierarchy_level, len(struct_id_path))
        id_cell = struct_id_path[level_hierarchy_chosen-1]
        acronym = atlas.structures.get(id_cell)['acronym']
        count_per_acronym_max_level[acronym] = count_per_acronym_max_level.get(acronym,0) + 1
    else:
        # Handle the case where structure ID is not found
        # For example, you can skip it or assign a default value
        count_per_acronym_max_level['OUTSIDE'] = count_per_acronym_max_level.get('OUTSIDE',0) + 1

print(count_per_acronym_max_level)

### Bar plot of cell count per region

If you run the previous cell with `max_hierarchy_level = 5` and then the next one, you will see that the majority of cells are within the `CTXpl` region.

In [ ]:
import matplotlib.pyplot as plt

# Sample dictionary
data = count_per_acronym_max_level

# Specify colors for each bar - the color of the atlas region (we set the outside label to black)
colors = [[color / 255 for color in atlas.structures[acronym]['rgb_triplet']] if atlas.structures.get(acronym) else [0,0,0] for acronym in data.keys()]#['red', 'green', 'blue']

# Extract categories and heights from the dictionary
categories = list(data.keys())
heights = list(data.values())

# Create a figure with a larger width
plt.figure(figsize=(10, 6))

# Create a bar chart
bars = plt.bar(categories, heights, color=colors)

# Rotate x-axis labels vertically
plt.xticks(rotation='vertical')

# Add labels and title
plt.xlabel('Brain region (hierarchy level '+str(max_hierarchy_level)+')')
plt.ylabel('# Cells')
plt.title('Cy3 positive cells per region')

# Display the plot
plt.show()

### Bar plot of cell count per region within a subregion, at a certain hierarchy level

With the cell below, we can break down the cell count within the subregions of the `CTXpl` region.

In [ ]:
# To break down what's happening in CTXpl

sub_region = 'CTXpl'
max_hierarchy_level = 7 # 1 = root, 2 = VS, fiber tracts, grey, etc.

id_sub_region = atlas.structures.get(sub_region)['id']

# Sometimes you don't want to get the finest subdivisions of the structures
# Here's a possible way of cropping at a certain hierarchy level

count_per_acronym_max_level_in_subregion = {}

for cell in cy3_pos_cells:
    struct_id = atlas.structure_from_coords(cell, microns=True) # We read the label image at the position of the cell
    structure_info = atlas.structures.get(struct_id)
    if structure_info:
        struct_id_path = structure_info["structure_id_path"]
        if id_sub_region in struct_id_path:
            level_hierarchy_chosen = min(max_hierarchy_level, len(struct_id_path))
            id_cell = struct_id_path[level_hierarchy_chosen-1]
            acronym = atlas.structures.get(id_cell)['acronym']
            count_per_acronym_max_level_in_subregion[acronym] = count_per_acronym_max_level_in_subregion.get(acronym,0) + 1
        else:
            count_per_acronym_max_level_in_subregion['OUTSIDE'] = count_per_acronym_max_level_in_subregion.get('OUTSIDE',0) + 1
    else:
        # Handle the case where structure ID is not found
        # For example, you can skip it or assign a default value
        count_per_acronym_max_level_in_subregion['OUTSIDE'] = count_per_acronym_max_level_in_subregion.get('OUTSIDE',0) + 1

print(count_per_acronym_max_level_in_subregion)


In [ ]:
import matplotlib.pyplot as plt

# Sample dictionary
data = count_per_acronym_max_level_in_subregion

# Specify colors for each bar - the color of the atlas region (we set the outside label to black)
colors = [[color / 255 for color in atlas.structures[acronym]['rgb_triplet']] if atlas.structures.get(acronym) else [0,0,0] for acronym in data.keys()]#['red', 'green', 'blue']

# Extract categories and heights from the dictionary
categories = list(data.keys())
heights = list(data.values())


# Create a figure with a larger width
plt.figure(figsize=(10, 6))

# Create a bar chart
bars = plt.bar(categories, heights, color=colors)

# Rotate x-axis labels vertically
plt.xticks(rotation='vertical')

# Add labels and title
plt.xlabel('Brain region (hierarchy level '+str(max_hierarchy_level)+')')
plt.ylabel('# Cells')
plt.title('Cy3 positive cells per region in subregion '+sub_region)

# Display the plot
plt.show()

# Extra

In [29]:
# Run this cells if you want to display the full atlas structure
atlas.structures

root (997)
├── VS (73)
│   ├── AQ (140)
│   ├── V3 (129)
│   ├── V4 (145)
│   │   └── V4r (153)
│   ├── VL (81)
│   │   ├── SEZ (98)
│   │   └── chpl (108)
│   └── c (164)
├── fiber tracts (1009)
│   ├── cbf (960)
│   │   ├── arb (728)
│   │   ├── cbc (744)
│   │   └── cbp (752)
│   │       ├── icp (1123)
│   │       │   └── sctd (553)
│   │       ├── mcp (78)
│   │       └── scp (326)
│   │           ├── dscp (812)
│   │           ├── sctv (866)
│   │           └── uf (850)
│   ├── cm (967)
│   │   ├── IIIn (832)
│   │   │   ├── mlf (62)
│   │   │   └── pc (158)
│   │   ├── IIn (848)
│   │   │   ├── bsc (916)
│   │   │   ├── csc (336)
│   │   │   ├── och (117)
│   │   │   └── opt (125)
│   │   ├── IVn (911)
│   │   ├── In (840)
│   │   │   ├── aco (900)
│   │   │   ├── lotg (21)
│   │   │   │   ├── lot (665)
│   │   │   │   └── lotd (538)
│   │   │   └── onl (1016)
│   │   ├── VIIIn (933)
│   │   │   ├── cVIIIn (948)
│   │   │   │   ├── bic (482)
│   │   │   │   ├── cic (633)
│   │   

In [30]:
atlas.structures.get('CA')

{'acronym': 'CA', 'id': 375, 'name': "Ammon's horn", 'structure_id_path': [997, 8, 567, 688, 695, 1089, 1080, 375], 'rgb_triplet': [126, 208, 75], 'mesh_filename': PosixPath('/home/gabri/.brainglobe/allen_mouse_25um_v1.2/meshes/375.obj'), 'mesh': None}

In [31]:
atlas.structures.get(996)['acronym'] # Just to know the syntax to collect the information

'AId1'

In [33]:
viewer.layers[1].scale

array([1., 1., 1.])

## Bash commands to setup this notebook locally
```
conda create -n brainrender-env python=3.9
conda activate brainrender-env
pip install jupyterlab
pip install brainrender-napari==0.0.3
pip install bg-atlasapi
pip install napari[all]
jupyter lab
```